In [1]:
#Module3 - Unit3 - Project3 - 'О Вкусной и здоровой пищще'

import pandas as pd
import re
import numpy as np
import datetime as dt
#импортировали исходные данные
df = pd.read_csv('main_task.csv')
df


,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963
...,...,...,...,...,...,...,...,...,...,...
39995,id_499,Milan,"['Italian', 'Vegetarian Friendly', 'Vegan Opti...",500.0,4.5,$$ - $$$,79.0,"[['The real Italian experience!', 'Wonderful f...",/Restaurant_Review-g187849-d2104414-Reviews-Ro...,d2104414
39996,id_6340,Paris,"['French', 'American', 'Bar', 'European', 'Veg...",6341.0,3.5,$$ - $$$,542.0,"[['Parisian atmosphere', 'Bit pricey but inter...",/Restaurant_Review-g187147-d1800036-Reviews-La...,d1800036
39997,id_1649,Stockholm,"['Japanese', 'Sushi']",1652.0,4.5,NaN,4.0,"[['Good by swedish standards', 'A hidden jewel...",/Restaurant_Review-g189852-d947615-Reviews-Sus...,d947615
39998,id_640,Warsaw,"['Polish', 'European', 'Eastern European', 'Ce...",641.0,4.0,$$ - $$$,70.0,"[['Underground restaurant', 'Oldest Restaurant...",/Restaurant_Review-g274856-d1100838-Reviews-Ho...,d1100838


In [2]:
#функции обработки

#считаем % пропусков по каждому признаку
#функция missing_data(data) будет принимать на вход датасет, и определять % пропусков и тип данных
def missing_data(data):
    for i in data.columns:
        md = 100-data[i].isna().value_counts()[0]*100/len(data[i])
        print('% пропусков в',i, md)
    return

#функция конвертации строковых данных в столбце Price Range, на фход идет серия, на выходе числовые значения 
#ghbpyfrf ценовой категории
def price_cat(data):
    if data == '$':
        return 1
    elif data == '$$ - $$$':
        return 2
    elif data == '$$$$':
        return 3

#функция которая получает на вход серию со списком всех кухонь и убирает аартефакты в названиях,
#формируя список 
def resplit(ct):
    ct = ct.replace(", ",'')
    ct = ct.replace("[",'')
    ct = ct.replace("]",'')
    ct = re.split("'(.*?)'",ct)
    ct = list(filter(None, ct))
    return ct

#функция разбивки типов кухонь , нужная для выполенния заданий в модуле
def split_cuisine(ct):
    if re.findall(r"'(.*?)'", ct) == 'Unknown':
        return 'Unknown'
    else:
        return  re.findall(r"'(.*?)'", ct)

#простая функция длинны списка
def len_list(data):
    return len(data)


# функиця разделения даты из отзывов
# тип данных строка, визуально похожая на списки
# необходимо разпарсить данные
# кроме того , в строке есть дата отзыва, ее тоже надо достать
# кол-во дат будет равно кол-ву отзывов

def split_date(ct):
    if re.findall(r'\d\d/\d\d/\d{4}', ct) == 'None':
        return dt.datetime(1900,12,31)
    else:
        return  re.findall(r'\d\d/\d\d/\d{4}', ct)
    
# функция возвращает разницу между первым и последним отзывом в формате datetime
def deltadays(rd):
    if len(rd)>=2:
        a = dt.datetime.strptime(rd[0], '%m/%d/%Y') - dt.datetime.strptime(rd[len(rd)-1], '%m/%d/%Y')
        return a
    else:
        return dt.timedelta(0)


# SCRAPING блок

In [3]:
#специально закомментирован т.к. данные уже получены и что .s каждый раз не ждать , импортируются отдельно


#скопируем часть данных в отдельный датафрейм для парсинга

#import requests
#from bs4 import BeautifulSoup
#import re
#import math
#import time

#df_cuisine_missing = df[['id','city','url_ta','cuisine']]

#display(df_cuisine_missing.head(3))


#Записать прогон циклом по NOR если он NAN тогда только брать урл
#данные обработали и выделили в отдельный файл, поэтому все закоментировано

#for i in range(len(df_cuisine_missing['cuisine'])-1,0,-1):
#    if df['cuisine'][i] == 'Unknown':
#  
#        url = 'https://www.tripadvisor.com' + df_cuisine_missing['url_ta'][i]
#        headers = {
#        'User-Agent': 'My User Agent 1.0',
#        'From': 'personal@domain.com'  # This is another valid field
#        }
#        response = requests.get(url, headers=headers)
#        soup = BeautifulSoup(response.text, 'lxml')
#        pattern = re.compile(r'>[^\d]\w+<')
#        pattern2 = re.compile(r'[^\d]\w+<')
#        pattern3 = re.compile(r'\w+')
#        
#        cuisine = soup.find_all('a', class_='_2mn01bsa')
#        cuis =[]
#        cuis = re.findall(pattern2, str(cuisine))
#        cuis = re.findall(pattern3, str(cuis))
#        df['col_cuisine'][i] = cuis
#    if i == 20000:
#        print('Пауза 5 минут',i)
#        time.sleep(300)
#    if i % 1000 == 0:
#        print('прошли тысячу строк',i)
#            
#df.to_csv('cuisine.csv')



# - парсинг Number of Reviews
#скопируем часть данных в отдельный датафрейм для парсинга
#import requests
#from bs4 import BeautifulSoup
#import re
#import math

#df_pars = df[['id','city','url_ta','nor']]

#df_pars
#df_pars['nor_new'] = 0

#Записать прогон циклом по NOR если он NAN тогда только брать урл
#данные обработали и выделили в отдельный файл, поэтому все закоментировано

#for i in range(0,len(df_pars['nor'])-1):
#    if pd.isnull(df_pars['nor'][i]) == True:
#        #контроль
#        print(i,'проверка',df_pars['nor'][i])
#        url = 'https://www.tripadvisor.com' + df_pars['url_ta'][i]
#        headers = {
#        'User-Agent': 'My User Agent 1.0',
#        'From': 'personal@domain.com'  # This is another valid field
#        }
#        response = requests.get(url, headers=headers)
#        soup = BeautifulSoup(response.text, 'lxml')
#        pattern = re.compile(r'"reviewCount":"\d*"')
#        pattern2 = re.compile(r'\d*')
#        rc = re.findall(pattern, str(soup))
#       
#        if isinstance(rc, str) == True:
#            print('проверка ',rc, type(rc))
#            df_pars['nor'][i] = re.findall(pattern2,rc[0])
#        else:
#            df_pars['nor'][i] = 0
#df.to_excel('df_pars.хls')

In [3]:
#смотрим все колонки, они же и будут признаками в этом датасете
display(df.columns)
#определяем какие признаки можно использовать

missing_data(df)
#Видим что 23% пропуска в Cuisine style 
#пропусков в Price Range 34.715
#пропусков в Number of Reviews 6.357500000000002

#смотрим визуально на данные
display(df.head(2))

Index(['Restaurant_id', 'City', 'Cuisine Style', 'Ranking', 'Rating',
       'Price Range', 'Number of Reviews', 'Reviews', 'URL_TA', 'ID_TA'],
      dtype='object')

% пропусков в Restaurant_id 0.0
% пропусков в City 0.0
% пропусков в Cuisine Style 23.207499999999996
% пропусков в Ranking 0.0
% пропусков в Rating 0.0
% пропусков в Price Range 34.715
% пропусков в Number of Reviews 6.357500000000002
% пропусков в Reviews 0.0
% пропусков в URL_TA 0.0
% пропусков в ID_TA 0.0


,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032


In [4]:
#Видим что надо убрать пропуски в признаках Cuisine Style,Price Range,Number of Reviews
#URL_TA ID_TA нас не сильно интересуют т.к. это ссылки на внешние ресурсы их можно использовать 
#для парсинга дополнительных данных что убрать пропуски в Number of Reviews, Cuisine_style

#- 0 - ПЕРВОНАЧАЛЬНЫЕ ЗАДАЧИ
#- 1 - необходимо получить список уникальных кухонь (сделано)
#- 2 - необходимо убрать пропуски в Price range     (сделано)
#- 3 - необходимо убрать пропуски Number of Reviews (сделано)
#- 4 - кол-во отзывов и разница между ними          (сделано)
#- 5 - дополнительные признаки                      (сделано)
#- 6 - парсинг для признака NOR                     (сделано)

In [5]:
# переименуем столбцы для удобства

df.rename(columns={'Restaurant_id': 'id',
                        'City': 'city',
                        'Cuisine Style': 'cuisine',
                        'Ranking': 'ranking',
                        'Rating': 'rating',
                        'Price Range': 'price_range',
                        'Number of Reviews': 'nor',
                        'Reviews': 'reviews',
                        'URL_TA': 'url_ta',
                        'ID_TA': 'id_ta' }, inplace=True)
display(df.head(2))

#созддим новый признак prcat(price category) скопируем данные из price_range
#и заменим модальным значением по этой категории 
df['prcat']=df['price_range']

#ghbvtyztv функцию замены признаков ценовой категории
df['prcat']=df['prcat'].apply(price_cat)

#заполняем пропуски на модальное значение 
df['prcat'].fillna(2, inplace=True, axis=0) #сделано на основе данных анализ из другого проектного ноутбука

#проверяем на наличие пропусков
missing_data(df)
#отлично prcat не имеет пропусков
#заменим данные в price_range на цисловые из prcat
df['price_range']=df['prcat']

display(df.head(5))

#мы изменили столбец price_range и создали новый prcat заполнив пропуски средним значением по городам , это 2
#можно удалить старый столбец
df.drop(['prcat'], inplace=True, axis=1)
df.head(2)


,id,city,cuisine,ranking,rating,price_range,nor,reviews,url_ta,id_ta
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032


% пропусков в id 0.0
% пропусков в city 0.0
% пропусков в cuisine 23.207499999999996
% пропусков в ranking 0.0
% пропусков в rating 0.0
% пропусков в price_range 34.715
% пропусков в nor 6.357500000000002
% пропусков в reviews 0.0
% пропусков в url_ta 0.0
% пропусков в id_ta 0.0
% пропусков в prcat 0.0


,id,city,cuisine,ranking,rating,price_range,nor,reviews,url_ta,id_ta,prcat
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,2.0,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,2.0
1,id_1535,Stockholm,NaN,1537.0,4.0,2.0,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,2.0
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,3.0,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,3.0
3,id_3456,Berlin,NaN,3458.0,5.0,2.0,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776,2.0
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,2.0,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963,2.0


,id,city,cuisine,ranking,rating,price_range,nor,reviews,url_ta,id_ta
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,2.0,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,2.0,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032


In [6]:
# - 1 - Получим список уникальных кухонь - СДЕЛАНО

df['cuisine'].head(5)
#заменим пропуски в типах кухнях на строковое выражение 'Unknown' 
#далее к этому выражению будет привязан процедура парсинга
df['cuisine'].fillna('Unknown', inplace=True)

cuisine_type = df['cuisine'].apply(resplit)
cuisine_type = cuisine_type.explode()

# создадим список уникальных кухонь
ctu = cuisine_type.value_counts().keys()

# удалим из списка значение Unknown т.к. это не кухня
ctu.drop('Unknown')



df.head(5)



,id,city,cuisine,ranking,rating,price_range,nor,reviews,url_ta,id_ta
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,2.0,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,Unknown,1537.0,4.0,2.0,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,3.0,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781
3,id_3456,Berlin,Unknown,3458.0,5.0,2.0,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,2.0,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963


In [7]:
#тут будем заполнять кухни  через внешний парсинг т.к. он долгий, потом просто импортируем данные 
# для этого была написана проценуда парсинга данных типа кухонь, т.к. процесс этот длительный, сделал
# его отдельно и выгрузил данные в файл для того что бы использовать через импорт , а не проводить каждый раз 
# парсинг 
# файл cuisine.csv
cuisine = pd.read_csv('cuisine.csv')
cuisine
df['cuisine']=cuisine['col_cuisine']

df



,id,city,cuisine,ranking,rating,price_range,nor,reviews,url_ta,id_ta
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,2.0,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,"['Asian', 'Nepali']",1537.0,4.0,2.0,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,3.0,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781
3,id_3456,Berlin,[],3458.0,5.0,2.0,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,2.0,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963
...,...,...,...,...,...,...,...,...,...,...
39995,id_499,Milan,"['Italian', 'Vegetarian Friendly', 'Vegan Opti...",500.0,4.5,2.0,79.0,"[['The real Italian experience!', 'Wonderful f...",/Restaurant_Review-g187849-d2104414-Reviews-Ro...,d2104414
39996,id_6340,Paris,"['French', 'American', 'Bar', 'European', 'Veg...",6341.0,3.5,2.0,542.0,"[['Parisian atmosphere', 'Bit pricey but inter...",/Restaurant_Review-g187147-d1800036-Reviews-La...,d1800036
39997,id_1649,Stockholm,"['Japanese', 'Sushi']",1652.0,4.5,2.0,4.0,"[['Good by swedish standards', 'A hidden jewel...",/Restaurant_Review-g189852-d947615-Reviews-Sus...,d947615
39998,id_640,Warsaw,"['Polish', 'European', 'Eastern European', 'Ce...",641.0,4.0,2.0,70.0,"[['Underground restaurant', 'Oldest Restaurant...",/Restaurant_Review-g274856-d1100838-Reviews-Ho...,d1100838


In [8]:
df['col_cuisine'] = df['cuisine'].apply(split_cuisine)
df['col_c'] = df['col_cuisine'].apply(len)
display(df.head(5))

#т.к. у нас все равно остались рестораны в котоорых не указано кухня, там стоит 0
# посчитаем сколько таких ресторанов что бы понять сколько у нас потерь данных

print('% пропусков в типах кухни ',(len(df[df['col_c']==0])*100)/(len(df)))

#уже лучше чем 23%

,id,city,cuisine,ranking,rating,price_range,nor,reviews,url_ta,id_ta,col_cuisine,col_c
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,2.0,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,"[European, French, International]",3
1,id_1535,Stockholm,"['Asian', 'Nepali']",1537.0,4.0,2.0,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,"[Asian, Nepali]",2
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,3.0,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,"[Japanese, Sushi, Asian, Grill, Vegetarian Fri...",7
3,id_3456,Berlin,[],3458.0,5.0,2.0,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776,[],0
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,2.0,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963,"[German, Central European, Vegetarian Friendly]",3


% пропусков в типах кухни  11.7075


In [5]:


#display(df.head(5))
#создадим отдельный датафрейм для того что бы вычислить самую модную кухню в городе

moda_c = df.explode('col_cuisine')

#moda_c.groupby('city', group_keys=False).apply(lambda x: x.nlargest(1, 'col_cuisine'))
#moda_c1 = moda_c[['city', 'col_cuisine']]
CM = df.explode('col_cuisine').groupby('city')['col_cuisine'].value_counts()

#display(CM)
CM1=pd.DataFrame(columns={'city','cuisine','count'})


#moda_c
#moda_c.groupby('city')['col_cuisine'].value_counts()

CM.values[0]
for i in range(0,len(CM)-1):
    new_row = {'city':list(CM.index[i])[0], 'cuisine':list(CM.index[i])[1], 'count':CM.values[i]} 
    CM1 = CM1.append(new_row, ignore_index=True)

city_list=[]
CM2 = pd.DataFrame()
CM1

CM1.groupby('city')['count'].max()
#CM1['city']
#for i in range(0,len(CM1)-1):
#    if CM1['city'][i] not in city_list:
#        city_list = city_list.append(CM1['city'][i])
#        display(city_list)
#        new_row = {'city':CM1['city'][i], 'cuisine':CM1['cuisine'][i], 'count':CM1['count'][i]}         
#        CM2 = CM2.append(new_row, ignore_index=True)
#CM2

display(df)

,id,city,cuisine,ranking,rating,nor,reviews,url_ta,id_ta,prcat,col_cuisine,col_c
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,2.0,"[European, French, International]",3
1,id_1535,Stockholm,Unknown,1537.0,4.0,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,2.0,[],0
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,3.0,"[Japanese, Sushi, Asian, Grill, Vegetarian Fri...",7
3,id_3456,Berlin,Unknown,3458.0,5.0,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776,2.0,[],0
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963,2.0,"[German, Central European, Vegetarian Friendly]",3
...,...,...,...,...,...,...,...,...,...,...,...,...
39995,id_499,Milan,"['Italian', 'Vegetarian Friendly', 'Vegan Opti...",500.0,4.5,79.0,"[['The real Italian experience!', 'Wonderful f...",/Restaurant_Review-g187849-d2104414-Reviews-Ro...,d2104414,2.0,"[Italian, Vegetarian Friendly, Vegan Options, ...",4
39996,id_6340,Paris,"['French', 'American', 'Bar', 'European', 'Veg...",6341.0,3.5,542.0,"[['Parisian atmosphere', 'Bit pricey but inter...",/Restaurant_Review-g187147-d1800036-Reviews-La...,d1800036,2.0,"[French, American, Bar, European, Vegetarian F...",5
39997,id_1649,Stockholm,"['Japanese', 'Sushi']",1652.0,4.5,4.0,"[['Good by swedish standards', 'A hidden jewel...",/Restaurant_Review-g189852-d947615-Reviews-Sus...,d947615,2.0,"[Japanese, Sushi]",2
39998,id_640,Warsaw,"['Polish', 'European', 'Eastern European', 'Ce...",641.0,4.0,70.0,"[['Underground restaurant', 'Oldest Restaurant...",/Restaurant_Review-g274856-d1100838-Reviews-Ho...,d1100838,2.0,"[Polish, European, Eastern European, Central E...",5


In [9]:
#объединение с полученными данными
df_nor = pd.read_excel('df_pars.xls')
#display(df['nor'][21])
#display(df_nor['nor'][21])
#df333 = df.merge(df_nor, how='right')
#display(df333['nor'][21])
#df333.drop(['Unnamed: 0'], axis=1, inplace= True)
df['nor']=df_nor['nor']
missing_data(df)
df

% пропусков в id 0.0
% пропусков в city 0.0
% пропусков в cuisine 0.0
% пропусков в ranking 0.0
% пропусков в rating 0.0
% пропусков в price_range 0.0
% пропусков в nor 0.0
% пропусков в reviews 0.0
% пропусков в url_ta 0.0
% пропусков в id_ta 0.0
% пропусков в col_cuisine 0.0
% пропусков в col_c 0.0


,id,city,cuisine,ranking,rating,price_range,nor,reviews,url_ta,id_ta,col_cuisine,col_c
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,2.0,194,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,"[European, French, International]",3
1,id_1535,Stockholm,"['Asian', 'Nepali']",1537.0,4.0,2.0,10,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,"[Asian, Nepali]",2
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,3.0,688,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,"[Japanese, Sushi, Asian, Grill, Vegetarian Fri...",7
3,id_3456,Berlin,[],3458.0,5.0,2.0,3,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776,[],0
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,2.0,84,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963,"[German, Central European, Vegetarian Friendly]",3
...,...,...,...,...,...,...,...,...,...,...,...,...
39995,id_499,Milan,"['Italian', 'Vegetarian Friendly', 'Vegan Opti...",500.0,4.5,2.0,79,"[['The real Italian experience!', 'Wonderful f...",/Restaurant_Review-g187849-d2104414-Reviews-Ro...,d2104414,"[Italian, Vegetarian Friendly, Vegan Options, ...",4
39996,id_6340,Paris,"['French', 'American', 'Bar', 'European', 'Veg...",6341.0,3.5,2.0,542,"[['Parisian atmosphere', 'Bit pricey but inter...",/Restaurant_Review-g187147-d1800036-Reviews-La...,d1800036,"[French, American, Bar, European, Vegetarian F...",5
39997,id_1649,Stockholm,"['Japanese', 'Sushi']",1652.0,4.5,2.0,4,"[['Good by swedish standards', 'A hidden jewel...",/Restaurant_Review-g189852-d947615-Reviews-Sus...,d947615,"[Japanese, Sushi]",2
39998,id_640,Warsaw,"['Polish', 'European', 'Eastern European', 'Ce...",641.0,4.0,2.0,70,"[['Underground restaurant', 'Oldest Restaurant...",/Restaurant_Review-g274856-d1100838-Reviews-Ho...,d1100838,"[Polish, European, Eastern European, Central E...",5


In [10]:

# - 4 - кол-во отзывов и разница между ними - СДЕЛАНО

#display(df.head(5))

df['reviews'].fillna('',inplace=True)
df['rev_date'] = df['reviews'].apply(split_date)


#создадим новый признак, кол-во отзывов
#lkz этого применим функцию считающую длинну списка с датами

df['col_r']=df['rev_date'].apply(len_list)

#для выполнения тестов по проекту необходимо посчитать самую большую разницу между двумя отзывами

#посчитали разницу между отзывами для всех ресторанов
df['time_delta'] = df['rev_date'].apply(deltadays)

#найдем теперь максимум
print('Максимальная разница между отзывами, дней', df['time_delta'].max())
print('среднее кол-во оценок относительно типа кухни и города')

#df.drop(['price_range', 'reviews', 'url_ta', 'id_ta', 'col_cuisine', 'rev_date', 'cuisine'], axis=1, inplace = True)
#df
#df['city'].groupby('col_r').mean()

Максимальная разница между отзывами, дней 3207 days 00:00:00
среднее кол-во оценок относительно типа кухни и города


In [11]:
# - 5 - 
# к дополнительным признакам можно добавить, кол-во мест в ресторане, 
# географический признак (район города, центр не центр)
# наличие дополнительных категориальных  метрик, типа вайфай, парковка, 
#

#============================================================================
#создадим отдельный дата фрейм с разбивкой по городам и айди ресторанов
#и диапазонам цены, что бы определить среднее для каждого города
#CPR city price range
#============================================================================

cpr = df[['city','price_range']]
display(cpr.value_counts())
display(cpr[cpr['city']=='London'].value_counts())
rpr = df[['id','price_range']]
display(rpr.value_counts())

from sklearn.preprocessing import LabelEncoder
#City encoding
le = LabelEncoder()
le.fit(df.city)
df['city_le']=le.transform(df.city)

df[15:25]

city        price_range
London      2.0            4518
Paris       2.0            4080
Madrid      2.0            2645
Barcelona   2.0            2165
Berlin      2.0            1776
                           ... 
Helsinki    3.0              10
Bratislava  3.0              10
Krakow      3.0              10
Luxembourg  3.0               9
Ljubljana   3.0               7
Length: 93, dtype: int64

city    price_range
London  2.0            4518
        1.0            1025
        3.0             214
dtype: int64

id       price_range
id_561   2.0            16
id_344   2.0            15
id_453   2.0            15
id_227   2.0            15
id_245   2.0            14
                        ..
id_7350  2.0             1
id_2179  3.0             1
id_735   1.0             1
id_218   3.0             1
id_9995  2.0             1
Length: 16535, dtype: int64

,id,city,cuisine,ranking,rating,price_range,nor,reviews,url_ta,id_ta,col_cuisine,col_c,rev_date,col_r,time_delta,city_le
15,id_1401,Dublin,['Cafe'],1405.0,4.0,2.0,28,"[['Lovely food', 'Great coffee'], ['12/01/2017...",/Restaurant_Review-g186605-d6633059-Reviews-Gr...,d6633059,[Cafe],1,"[12/01/2017, 09/14/2017]",2,78 days,8
16,id_5257,Berlin,"['Japanese', 'Asian', 'Thai', 'Vietnamese']",5259.0,4.0,2.0,3,"[['This is a real hidden Sushi-gem'], ['04/12/...",/Restaurant_Review-g187323-d10266473-Reviews-A...,d10266473,"[Japanese, Asian, Thai, Vietnamese]",4,[04/12/2017],1,0 days,3
17,id_11121,London,"['Bar', 'Pub']",11131.0,4.5,2.0,7,"[['gin tour venue', 'Gin Tour'], ['02/28/2017'...",/Restaurant_Review-g186338-d8659568-Reviews-46...,d8659568,"[Bar, Pub]",2,"[02/28/2017, 10/31/2016]",2,120 days,16
18,id_3127,Berlin,['German'],3129.0,4.0,2.0,12,"[['Nice stay, very good hotel', 'great residen...",/Restaurant_Review-g187323-d3702312-Reviews-Al...,d3702312,[German],1,"[08/16/2017, 10/21/2016]",2,299 days,3
19,id_2791,Brussels,['Chinese'],2792.0,3.0,2.0,12,"[['Seriously, is this a restaurant?', 'Not tha...",/Restaurant_Review-g947638-d8155327-Reviews-Ho...,d8155327,[Chinese],1,"[04/12/2016, 08/13/2015]",2,243 days,5
20,id_368,Munich,"['German', 'Vegetarian Friendly']",370.0,5.0,2.0,27,"[['Hidden Gem', 'Traditional Bavarian restaura...",/Restaurant_Review-g187309-d10524693-Reviews-S...,d10524693,"[German, Vegetarian Friendly]",2,"[12/12/2017, 12/04/2017]",2,8 days,21
21,id_5844,Madrid,[],5847.0,4.0,2.0,0,"[[], []]",/Restaurant_Review-g187514-d10058810-Reviews-B...,d10058810,[],0,[],0,0 days,19
22,id_13016,London,"['Bar', 'British', 'Pub']",13026.0,3.5,2.0,52,"[['Nice pub but not for food', 'Recently renov...",/Restaurant_Review-g186338-d2508041-Reviews-Th...,d2508041,"[Bar, British, Pub]",3,"[08/05/2017, 04/29/2017]",2,98 days,16
23,id_72,Zurich,"['Asian', 'Thai', 'Vegetarian Friendly', 'Glut...",73.0,4.0,2.0,371,"[['Great Thai Food', 'Great Thai Food In The M...",/Restaurant_Review-g188113-d1018940-Reviews-Wh...,d1018940,"[Asian, Thai, Vegetarian Friendly, Gluten Free...",5,"[12/01/2017, 10/05/2017]",2,57 days,30
24,id_14973,London,"['Cafe', 'British']",14984.0,3.0,1.0,18,"[['Fast and hot', 'Reasonable'], ['08/06/2017'...",/Restaurant_Review-g186338-d7617571-Reviews-Sq...,d7617571,"[Cafe, British]",2,"[08/06/2017, 05/09/2017]",2,89 days,16


In [12]:
# - 6- удалим ненужные текстовые признаки и посмотрим пропуски

#df.drop(['cuisine','price_range', 'reviews', 'url_ta', 'id_ta', 'col_cuisine', 'rev_date'], axis=1, inplace = True)
#df.head(5)


df.to_csv('final.csv')